In [ ]:
# ✅ Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ Dataset path
DATASET_DIR = '/content/drive/MyDrive/dataset_cancer_v1/classificacao_binaria/100X'

# ✅ Imports
import os
import numpy as np
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization, Activation, Concatenate, AveragePooling2D
from tensorflow.keras.optimizers import Adam
import random

# === CONFIG ===
IMG_SIZE = (224, 224)  # Standard for DenseNet
EPOCHS = 10
BATCH_SIZE = 32
LEARNING_RATE = 0.0001

In [ ]:
# === DenseNet Building Blocks ===
def dense_block(x, blocks, growth_rate, name):
    """A dense block."""
    for i in range(blocks):
        x = conv_block(x, growth_rate, name=f'{name}_block{i + 1}')
    return x

def transition_block(x, reduction, name):
    """A transition block."""
    in_channels = x.shape[-1]
    x = BatchNormalization(name=f'{name}_bn')(x)
    x = Activation('relu', name=f'{name}_relu')(x)
    x = Conv2D(int(in_channels * reduction), 1, use_bias=False, name=f'{name}_conv')(x)
    x = AveragePooling2D(2, strides=2, name=f'{name}_pool')(x)
    return x

def conv_block(x, growth_rate, name):
    """A building block for a dense block."""
    x1 = BatchNormalization(name=f'{name}_bn1')(x)
    x1 = Activation('relu', name=f'{name}_relu1')(x1)
    x1 = Conv2D(4 * growth_rate, 1, use_bias=False, name=f'{name}_conv1')(x1)
    x1 = BatchNormalization(name=f'{name}_bn2')(x1)
    x1 = Activation('relu', name=f'{name}_relu2')(x1)
    x1 = Conv2D(growth_rate, 3, padding='same', use_bias=False, name=f'{name}_conv2')(x1)
    x = Concatenate(name=f'{name}_concat')([x, x1])
    return x

In [ ]:
# === Visualize Sample Images ===
def plot_sample_images(dataset_path, num_images_per_class=5):
    classes = sorted(os.listdir(dataset_path))
    # plt.figure(figsize=(15, len(classes) * 2.5))
    for class_idx, class_name in enumerate(classes):
        class_path = os.path.join(dataset_path, class_name)
        images = [img for img in os.listdir(class_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
        sample_images = random.sample(images, min(num_images_per_class, len(images)))
        for i, image_name in enumerate(sample_images):
            img_path = os.path.join(class_path, image_name)
            img = load_img(img_path, target_size=IMG_SIZE)
            # plt.subplot(len(classes), num_images_per_class, class_idx * num_images_per_class + i + 1)
            # plt.imshow(img)
            # plt.axis('off')
            # plt.title(class_name)
    # plt.tight_layout()
    # plt.show()

# === Load Dataset ===
def load_dataset(path):
    images = []
    labels = []
    for label_name in os.listdir(path):
        label_folder = os.path.join(path, label_name)
        if os.path.isdir(label_folder):
            for fname in os.listdir(label_folder):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(label_folder, fname)
                    img = load_img(img_path, target_size=IMG_SIZE)
                    img = img_to_array(img) / 255.0
                    images.append(img)
                    labels.append(label_name)
    return np.array(images), np.array(labels)

In [ ]:
# === Load and Prepare Data ===
print("🔄 Loading dataset...")
X, y = load_dataset(DATASET_DIR)
print(f"✅ Loaded {len(X)} images.")

# === Visualize ===
print("🖼 Displaying sample images...")
plot_sample_images(DATASET_DIR)

In [ ]:
# === Encode Labels ===
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
loss_fn = 'binary_crossentropy'
activation_fn = 'sigmoid'
output_units = 1

# === Split Dataset ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

In [ ]:
# === Build DenseNet Model ===
def build_densenet_model(input_shape=(224, 224, 3), num_classes=1, growth_rate=32, blocks=[6, 12, 24, 16]):
    inputs = Input(shape=input_shape)
    
    # Initial convolution
    x = Conv2D(64, 7, strides=2, padding='same', use_bias=False, name='conv1')(inputs)
    x = BatchNormalization(name='bn1')(x)
    x = Activation('relu', name='relu1')(x)
    x = MaxPooling2D(3, strides=2, padding='same', name='maxpool1')(x)
    
    # Dense blocks
    for i, block in enumerate(blocks):
        x = dense_block(x, block, growth_rate, name=f'dense_block{i + 1}')
        if i != len(blocks) - 1:
            x = transition_block(x, 0.5, name=f'transition_block{i + 1}')
    
    # Final batch norm and activation
    x = BatchNormalization(name='bn_final')(x)
    x = Activation('relu', name='relu_final')(x)
    
    # Global average pooling
    x = AveragePooling2D(pool_size=(7, 7), name='avgpool')(x)
    x = Flatten(name='flatten')(x)
    
    # Fully connected layers
    x = Dense(512, activation='relu', name='fc1')(x)
    x = Dropout(0.5, name='dropout1')(x)
    x = Dense(256, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='dropout2')(x)
    outputs = Dense(num_classes, activation=activation_fn, name='predictions')(x)
    
    model = Model(inputs, outputs, name='DenseNet')
    return model

# Create model
model = build_densenet_model(input_shape=(*IMG_SIZE, 3), num_classes=output_units)

# Compile
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss=loss_fn, metrics=['accuracy'])
model.summary()

In [ ]:
# === Train Model ===
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)

In [ ]:
# === Evaluate ===
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"📊 Test Accuracy: {test_acc:.2f}")

In [ ]:
# === Prediction Function ===
def predict_image(img_path):
    img = load_img(img_path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    label = "Malignant" if prediction[0][0] > 0.5 else "Benign"
    confidence = prediction[0][0] if prediction[0][0] > 0.5 else 1 - prediction[0][0]

    print(f"Prediction: {label} ({confidence:.2f})")
    # plt.imshow(load_img(img_path))
    # plt.title(f"{label} ({confidence:.2f})")
    # plt.axis('off')
    # plt.show()

# Example:
# predict_image('/content/drive/MyDrive/dataset_cancer_v1/classificacao_binaria/100X/benign/img001.png')